Implementar los problemas 3, 4 y 6 de la práctica 2 utilizando Keras.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.datasets import cifar10

%matplotlib inline

In [12]:
(Xtrain, ytrain), (Xtest, ytest) = cifar10.load_data()

ytrain = ytrain[:,0]
ytest = ytest[:,0]

Xtrain = Xtrain.astype(float)
Xtest = Xtest.astype(float)

m = Xtrain.shape[0]
n = Xtrain[0].size

mu = np.mean(Xtrain, axis=0)
std = np.std(Xtrain, axis=0)

Xtrain -= mu
Xtrain /= std
Xtest -= mu
Xtest /= std

Xtrain = np.reshape(Xtrain, (m, n))
Xtest = np.reshape(Xtest, (Xtest.shape[0],))

# summarize loaded dataset
print('Train: X=%s, y=%s' % (Xtrain.shape, ytrain.shape))
print('Test: X=%s, y=%s' % (Xtest.shape, ytest.shape))

KeyboardInterrupt: 

: 

In [6]:
def accuracy(ytrue, ypred):
    ypred_idx = np.argmax(ypred, axis=1)

    return np.mean(ypred_idx==ytrue)

In [ ]:
model = tf.keras.models.Model()
opt = tf.keras.optimizers.SGD(learning_rate=.003)
input = tf.keras.layers.Dense(1, input_shape=(n,), activation='linear', use_bias=True)
model.summary()